In [ ]:
%env CUDA_LAUNCH_BLOCKING=1
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext autoreload
%autoreload 2
import glob
import tqdm
import matplotlib.pyplot as plt
from GraPL.evaluate import bsds_score
from GraPL import get_DINO_embeddings
from skimage.segmentation import slic
import numpy as np
import json
import os
import warnings
import torch
warnings.filterwarnings("ignore")

In [ ]:
image_paths = glob.glob("datasets/BSDS500/BSDS500/data/images/test/*.jpg")

num_trials = 10

with tqdm.tqdm(total=num_trials*len(image_paths)) as pbar:
    for trial_num in range(num_trials):
        os.makedirs(f"experiment_results/baselines/dino_slic/{trial_num}", exist_ok=True)
        paramset_scores = {}
        for image_path in image_paths:
            id = image_path.split("/")[-1].split(".")[0]
            image = plt.imread(image_path)
            image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float().to("mps")
            embeddings,_ = get_DINO_embeddings(image, image.shape[2] // 14, dimensions=3)
            embeddings += embeddings.min()
            embeddings /= embeddings.max()
            embeddings *= 255
            embeddings = embeddings.cpu().numpy().astype(np.uint8)
            seg = slic(embeddings, n_segments=14, compactness=0.1, sigma=10)
            plt.imsave(f"experiment_results/baselines/dino_slic/{trial_num}/{id}.png", seg, cmap="viridis")
            image_scores = bsds_score(id, f"experiment_results/baselines/dino_slic/{trial_num}/{id}.png")
            paramset_scores[id] = image_scores
            pbar.update(1)
        with open(f'experiment_results/baselines/dino_slic/{trial_num}/scores.json', 'w') as fp:
            results = {"hyperparams": "slic segmentation by dino embeddings with compactness=0.1 and sigma=10", "scores": paramset_scores}
            json.dump(results, fp)